In [ ]:
%pip  install --upgrade transformers optimum

In [ ]:
%pip install torch

In [ ]:
# %pip install -U wheel

In [ ]:
# %pip install --upgrade auto-gptq

In [ ]:
# %pip install -v flash-attn==2.1.1 --no-build-isolation

In [ ]:
import torch
print(torch.__version__)
print (torch.cuda.is_available())

In [ ]:
%pip install accelerate

In [ ]:
%pip install --upgrade bitsandbytes

In [ ]:
%pip install --upgrade scipy

In [ ]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"


In [ ]:
%pip install ipywidgets
# from .autonotebook import tqdm as notebook_tqdm

In [ ]:
%pip install --upgrade setuptools

In [ ]:
from transformers import BitsAndBytesConfig
# compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
)

In [ ]:
from transformers import AutoModelForCausalLM, pipeline

model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map={"": 0}
)
# # model_name_or_path = "TheBloke/dolphin-2.7-mixtral-8x7b-GPTQ"
# model_name_or_path = "mistralai/Mixtral-8x7B-Instruct-v0.1"
# # To use a different branch, change revision
# # For example: revision="gptq-4bit-128g-actorder_True"
# model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
#                                                device_map="auto",
#                                                torch_dtype=torch.float16,
#                                                offload_folder="offload",
#                                                trust_remote_code=True,
#                                                low_cpu_mem_usage=True,
#                                                quantization_config=bnb_config)
                                             

# tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)



In [ ]:
print(model)


In [ ]:
from transformers import  AutoTokenizer

#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id =  tokenizer.unk_token_id
tokenizer.padding_side = 'left'

In [ ]:
prompt="Summarize your thoughts on AI hallucinations"

In [ ]:
input_ids = tokenizer.encode(prompt, return_tensors='pt',padding=True).cuda()
output = model.generate(inputs=input_ids, pad_token_id=tokenizer.unk_token_id,do_sample=True,temperature=0.4, repetition_penalty=1.1, top_p=0.95, top_k=40, max_new_tokens=512).cuda()
print(tokenizer.decode(output[0]))

In [ ]:
print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    pad_token_id=tokenizer.unk_token_id,
    do_sample=True,
    temperature=0.4,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt)[0]['generated_text'])

In [ ]:
messages = [
    {"role": "user", "content": "Tell me a recipe for Upma."},
]

print("\n\n*** Prompt:")
input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    return_tensors="pt",
)
print(tokenizer.decode(input_ids[0]))

print("\n\n*** Generate:")
with torch.autocast("cuda", dtype=torch.bfloat16):
    output = model.generate(
        input_ids=input_ids.to("cuda"),
        max_new_tokens=1024,
        return_dict_in_generate=True,
    )

response = tokenizer.decode(
    output["sequences"][0][len(input_ids[0]):], 
    skip_special_tokens=True
)
print(response)
